## Overview

This notebook is intended to:
- Demonstrate how to access time series data for meters and SCADAs using Awesense's Energy Data Model (EDM) SQL API.

Please refer to the [main_concepts.ipynb](../2_main_concepts/main_concepts.ipynb) notebook for a high-level introduction to and simpler examples of the core views and functions available in Awesense's Energy Data Model (EDM).

---

## Set up

In [1]:
import getpass
import plotly.express as px
import plotly.io as pio
import time
import urllib.parse
import pandas as pd

# Allow charts to persist between notebook sessions.
pio.renderers.default='notebook'

**Connection**

Enter the EDM server address and the login credentials provided by Awesense. If you do not have the credentials, or have any trouble connecting, please contact api@awesense.com.
<span style='color:red'> **Please do NOT store the credentials in the notebook, nor share them with anyone.** </span>

In [2]:
edm_address = getpass.getpass(prompt='EDM server address: ')

print('\nEDM login information')
edm_name = getpass.getpass(prompt='Username: ')
edm_password = getpass.getpass(prompt='Password: ')
edm_password = urllib.parse.quote(edm_password)

%load_ext sql
%sql postgresql://$edm_name:$edm_password@$edm_address/edm
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

# Delete the credential variables for security purpose.
del edm_name, edm_password

EDM server address: ········

EDM login information
Username: ········
Password: ········


---

## Time Series
Various temporary views are created below to optimize query performance within the notebook.

### Meter Time Series

*High level approach*
- Create a temporary view `grid_element_metric`, containing information on meters for grid elements and convenient access to the actual data.
- The `grid_get_downstream()` function is used to gather information of meters that are downstream of a specified grid element.
- Time series are retrieved using the `ts_data_source_select()` function.

**Downstream of a Grid** 
- Specify `grid_id` and `grid_element_id` whose downstream meter data to fetch for.

In [43]:
grid_id = input('Grid Id: ') # awefice
grid_element_id = input('Grid Element Id: ') # line_segment_57

Grid Id: North Central Zone
Grid Element Id: 12373_hvmv


In [440]:
grid_id = 'North Central Zone'
grid_element_id = '1213'

In [446]:
grid_id = 'awefice'
grid_element_id = 'transformer_6'

Check when this grid was last updated.

In [538]:
%%sql

SELECT last_updated
FROM grid
WHERE grid_id = '{grid_id}';

last_updated
2021-01-01 20:00:00+00:00


Finding the first branching after the first element - grid_element_id

In [447]:
Tree = %sql SELECT grid_element_id, terminal1_cn, terminal2_cn FROM grid_get_downstream('{grid_id}', '{grid_element_id}');
Tree = Tree.DataFrame()

AA = Tree[Tree['terminal1_cn']==Tree[Tree['grid_element_id']==grid_element_id]['terminal2_cn'].iloc[0]]
if (AA['terminal1_cn'].value_counts()>1).iloc[0]:
    print(f"First branching includes {AA}")
else:
    while ((AA['terminal1_cn'].value_counts()==1).iloc[0]):
        AA = Tree[Tree['terminal1_cn']==AA['terminal2_cn'].iloc[0]]
        if (AA['terminal1_cn'].value_counts()>1).iloc[0]:
                print(f"First branching includes {AA}")
                break
                


First branching includes      grid_element_id terminal1_cn terminal2_cn
36  line_segment_136      con_241      con_350
58   line_segment_89      con_241      con_242


In [546]:
GG = %sql select * from grid_get_downstream('{grid_id}', '{grid_element_id}') where terminal1_cn='VCN.L7-97438';
GG

grid_id,grid_element_id,type,customer_type,phases,is_underground,is_producer,is_consumer,is_switchable,switch_is_open,terminal1_cn,terminal2_cn,power_flow_direction,upstream_grid_element_id,geometry,meta
North Central Zone,26033,Transformer,Transformer,B,None,False,False,False,None,VCN.L7-97438,VCN.26033,1to2,None,0101000020E610000025ECC776CFA254C0AA9253A69A3A4340,"{'name': '26033', 'label': '7Y53', 'latitude': None, 'load_loss': 1116, 'longitude': None, 'mv_feeder': '0601', 'ownership': 'utility', 'make_model': None, 'rating_kva': 15.0, 'enclosure_id': None, 'no_load_loss': 193, 'voltage_level': 'MV/LV', 'commission_date': '1997-12-12', 'primary_voltage': 14000, 'maintenance_type': None, 'manufacture_date': '1996-10-06', 'installation_type': None, 'secondary_voltage': None, 'hvmv_parent_element': None, 'is_operation_status': None, 'parent_transformer_id': None, 'winding_configuration': None}"
North Central Zone,L7-97440,ACLineSegment,arc:PRIMARY_CONDUCTOR,ABC,None,False,False,False,None,VCN.L7-97438,VCN.L7-97440,both,None,0102000020E6100000020000009686FA17D0A254C04FB0F6347F3A434025ECC776CFA254C0AA9253A69A3A4340,"{'name': '97440', 'label': 'L7', 'system': None, 'rating_a': 529, 'materials': 'Al', 'latitude_1': None, 'latitude_2': None, 'make_model': '336 MCM ACSR', 'line_length': None, 'longitude_1': None, 'longitude_2': None, 'is_insulated': None, 'ph_ph_voltage': 14000, 'voltage_level': 'MV', 'wire_diameter': 18.288, 'hvmv_parent_element': None, 'is_operation_status': True, 'mvlv_parent_element': None, 'is_ground_deployable': None}"


In [621]:
grid_id = 'North Central Zone'
grid_element_id = 'L7-97438'

In [710]:
#Finding branches in NCZ and only in ABC lines

Tree = %sql SELECT grid_element_id, phases, terminal1_cn, terminal2_cn FROM grid_get_downstream('{grid_id}', '{grid_element_id}');
Tree = Tree.DataFrame()

#Make sure first element has ABC phases
if sum(Tree[Tree['grid_element_id']==grid_element_id]['phases'] != 'ABC')==1:
    print(f'{grid_element_id} is not ABC')

#Find the next link
AA = Tree[Tree['terminal1_cn']==Tree[Tree['grid_element_id']==grid_element_id]['terminal2_cn'].iloc[0]]

#If there are at least 2 ABC links connected, then break
if (sum(AA['phases']=='ABC')==2):
    print(f"First branching includes {AA}")

else:
    # As long as there is only one link
    while (sum(AA['phases']=='ABC')==1):
        
        #Go on to the next link
        for jj in range(0,len(AA['terminal1_cn'])):
            if AA['phases'].iloc[jj]=='ABC':
                BB = Tree[(Tree['terminal1_cn']==AA['terminal2_cn'].iloc[jj])]

        AA = BB
        
#If there are at least 2 ABC links connected, then break

        if (sum(AA['phases']=='ABC')==2):
            print(f"First branching includes {AA}")

                


First branching includes     grid_element_id phases   terminal1_cn   terminal2_cn
526       L7-100128    ABC  VCN.L7-100134  VCN.L7-100128
691        L7-37052    ABC  VCN.L7-100134   VCN.L7-37052


Finding single\double phase clusters that diverge from a triple phase line

In [340]:
grid_id = 'awefice'
grid_element_id = 'transformer_2'

#Retrieving elements, connections and phases from all elements downstream
Tree2 = %sql SELECT grid_element_id, terminal1_cn, terminal2_cn, phases FROM grid_get_downstream('{grid_id}', '{grid_element_id}');
Tree2 = Tree2.DataFrame()

#Merging elements that come one following the other
BB = Tree2.merge(Tree2, left_on='terminal2_cn', right_on='terminal1_cn',suffixes=('_upper', '_lower'))

#Tracing elements where the phases change from ABC to non-ABC
bif = BB[(BB['phases_upper']=='ABC') & (BB['phases_lower']!='ABC')]
bif = bif.reset_index()


,index,grid_element_id_upper,terminal1_cn_upper,terminal2_cn_upper,phases_upper,grid_element_id_lower,terminal1_cn_lower,terminal2_cn_lower,phases_lower
0,1930,L25A-144564,VCN.L25A-144626,VCN.L25A-144564,ABC,L25A-135841,VCN.L25A-144564,VCN.L25A-135841,B
1,1933,L25A-144573,VCN.L25A-144564,VCN.L25A-144573,ABC,L25A-136790,VCN.L25A-144573,VCN.L25A-136790,B
2,1940,L25A-144604,VCN.L25A-144706,VCN.L25A-144604,ABC,L25A-116071,VCN.L25A-144604,VCN.L25A-116071,B
3,1945,L25A-144623,VCN.L25A-144700,VCN.L25A-144623,ABC,L25A-135865,VCN.L25A-144623,VCN.L25A-135865,B
4,1952,L25A-144655,VCN.L25A-144573,VCN.L25A-144655,ABC,23794,VCN.L25A-144655,VCN.23794,C


Finding number of meters and the sum of their consumption in each cluster beneath ABC elements

In [428]:
grid_id = 'North Central Zone'
grid_element_id = 'L17-57698'

#Retrieving elements, connections and phases from all elements downstream
Tree2 = %sql SELECT grid_element_id, terminal1_cn, terminal2_cn, phases FROM grid_get_downstream('{grid_id}', '{grid_element_id}');
Tree2 = Tree2.DataFrame()

#Merging elements that come one following the other
BB = Tree2.merge(Tree2, left_on='terminal2_cn', right_on='terminal1_cn',suffixes=('_upper', '_lower'))

#Tracing elements where the phases change from ABC to non-ABC
bif = BB[(BB['phases_upper']=='ABC') & (BB['phases_lower']!='ABC')]
bif['Meters_in_cluster'] = 0
bif['Total_consumption'] = 0
bif = bif.reset_index()

#Calculating consumption and munber of meters in each cluster
for ii in range(0,len(bif['grid_element_id_lower'])):
#    print(bif['grid_element_id_lower'].iloc[ii])
    grid_element_id = bif['grid_element_id_lower'].iloc[ii]
    
    EE = %sql CREATE OR REPLACE TEMPORARY VIEW grid_element_metric AS SELECT grid_id, grid_element_id, phases, type, provider, direction, friendly_id, metric_key AS metric, valid, timestamp, value \
    FROM grid_element_data_source geds \
    JOIN UNNEST(geds.metrics::TEXT[]) AS metric_key \
        ON true \
    LEFT JOIN ts_data_source_select(grid_element_data_source_id, metric_key) AS ts \
        ON true;

    FF = %sql CREATE OR REPLACE TEMPORARY VIEW meter_data_source2 AS \
        SELECT meter.grid_id,\
                meter.grid_element_id,\
                geds.grid_element_data_source_id,\
                geds.friendly_id,\
                geds.phases,\
                geds.provider,\
                metric_key as metric,\
                lower(geds.valid) as start_time,\
                upper(geds.valid) as end_time\
        FROM grid_get_downstream('{grid_id}', '{grid_element_id}') AS meter\
        LEFT JOIN grid_element_data_source geds\
            ON meter.grid_element_id = geds.grid_element_id\
            AND meter.grid_id = geds.grid_id\
            AND geds.type = 'CONSUMER'\
        JOIN UNNEST(geds.metrics::TEXT[]) AS metric_key\
            ON true\
        WHERE meter.type = 'Meter';

    GG = %sql CREATE OR REPLACE TEMPORARY VIEW meter_consumption2 AS\
    SELECT meter.grid_id,\
            meter.grid_element_id,\
            meter.friendly_id,\
            timestamp,\
            value AS kWh,\
            meter.phases\
    FROM meter_data_source2 meter\
    LEFT JOIN grid_element_metric gem\
        ON gem.grid_id = meter.grid_id\
        AND gem.grid_element_id = meter.grid_element_id\
    WHERE gem.metric = 'kWh'\
       AND gem.type = 'CONSUMER';

    II = %sql CREATE OR REPLACE TEMPORARY VIEW ts_stats2 AS\
        SELECT SUM(kWh) AS kWh, MIN(timestamp) AS start_timerange, MAX(timestamp) AS end_timerange\
        FROM meter_consumption2;

    JJ = %sql select COUNT(DISTINCT grid_element_id) FROM meter_data_source2;
    KK = %sql SELECT SUM(kWh) AS kWh FROM meter_consumption2;
    
    bif.loc[ii,'Meters_in_cluster'] = JJ[0][0]
    bif.loc[ii,'Total_consumption'] = KK[0][0]
    
bif.head()

/var/folders/03/68xx11110r9b4mpdz752w79r0000gn/T/ipykernel_24179/3590155273.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/03/68xx11110r9b4mpdz752w79r0000gn/T/ipykernel_24179/3590155273.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/03/68xx11110r9b4mpdz752w79r0000gn/T/ipykernel_24179/3590155273.py:34: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '79588.15406465203' has dtype incompatible

,index,grid_element_id_upper,terminal1_cn_upper,terminal2_cn_upper,phases_upper,grid_element_id_lower,terminal1_cn_lower,terminal2_cn_lower,phases_lower,Meters_in_cluster,Total_consumption
0,422,2431,VCN.2135,VCN.2431,ABC,17Y73,VCN.2431,VCN.17Y73,C,1,79588.154065
1,1067,L17-57698,VCN.L17-57696,VCN.L17-57698,ABC,L17-138308,VCN.L17-57698,VCN.L17-138308,C,3,361528.136421
2,1068,L17-57698,VCN.L17-57696,VCN.L17-57698,ABC,L17-138359,VCN.L17-57698,VCN.L17-138359,A,2,163331.144778
3,1081,L17-57708,VCN.L17-57707,VCN.L17-57708,ABC,L17-136862,VCN.L17-57708,VCN.L17-136862,C,6,530320.101787
4,1088,L17-57716,VCN.L17-57715,VCN.L17-57716,ABC,L17-136043,VCN.L17-57716,VCN.L17-136043,C,3,318897.525233


Finding clusters beneath ABC elements and the sum of the consumption per timestamp

In [ ]:
grid_id = 'North Central Zone'
grid_element_id = 'L17-57698'

#Retrieving elements, connections and phases from all elements downstream
Tree2 = %sql SELECT grid_element_id, terminal1_cn, terminal2_cn, phases FROM grid_get_downstream('{grid_id}', '{grid_element_id}');
Tree2 = Tree2.DataFrame()

#Merging elements that come one following the other
BB = Tree2.merge(Tree2, left_on='terminal2_cn', right_on='terminal1_cn',suffixes=('_upper', '_lower'))

#Tracing elements where the phases change from ABC to non-ABC
bif = BB[(BB['phases_upper']=='ABC') & (BB['phases_lower']!='ABC')]
bif['Meters_in_cluster'] = 0
bif['Total_consumption'] = 0
bif = bif.reset_index()

#Calculating consumption and munber of meters in each cluster
#for ii in range(0,len(bif['grid_element_id_lower'])):
for ii in range(0,len(bif['grid_element_id_lower'])):
    grid_element_id = bif['grid_element_id_lower'].iloc[ii]
    
    EE = %sql CREATE OR REPLACE TEMPORARY VIEW grid_element_metric AS SELECT grid_id, grid_element_id, phases, type, provider, direction, friendly_id, metric_key AS metric, valid, timestamp, value \
    FROM grid_element_data_source geds \
    JOIN UNNEST(geds.metrics::TEXT[]) AS metric_key \
        ON true \
    LEFT JOIN ts_data_source_select(grid_element_data_source_id, metric_key) AS ts \
        ON true;

    FF = %sql CREATE OR REPLACE TEMPORARY VIEW meter_data_source2 AS \
        SELECT meter.grid_id,\
                meter.grid_element_id,\
                geds.grid_element_data_source_id,\
                geds.friendly_id,\
                geds.phases,\
                geds.provider,\
                metric_key as metric,\
                lower(geds.valid) as start_time,\
                upper(geds.valid) as end_time\
        FROM grid_get_downstream('{grid_id}', '{grid_element_id}') AS meter\
        LEFT JOIN grid_element_data_source geds\
            ON meter.grid_element_id = geds.grid_element_id\
            AND meter.grid_id = geds.grid_id\
            AND geds.type = 'CONSUMER'\
        JOIN UNNEST(geds.metrics::TEXT[]) AS metric_key\
            ON true\
        WHERE meter.type = 'Meter';
    
    %sql DROP VIEW IF EXISTS meter_consumption2 CASCADE;
    
    GG = %sql CREATE OR REPLACE TEMPORARY VIEW meter_consumption2 AS\
    SELECT meter.grid_id,\
            meter.grid_element_id,\
            meter.friendly_id,\
            timestamp,\
            value AS kWh,\
            meter.phases AS Phase\
    FROM meter_data_source2 meter\
    LEFT JOIN grid_element_metric gem\
        ON gem.grid_id = meter.grid_id\
        AND gem.grid_element_id = meter.grid_element_id\
    WHERE gem.metric = 'kWh'\
       AND gem.type = 'CONSUMER';

    II = %sql CREATE OR REPLACE TEMPORARY VIEW ts_stats2 AS\
        SELECT SUM(kWh) AS kWh, MIN(timestamp) AS start_timerange, MAX(timestamp) AS end_timerange\
        FROM meter_consumption2;

    JJ = %sql select COUNT(DISTINCT grid_element_id) FROM meter_data_source2;
    KK = %sql SELECT SUM(kWh) AS kWh FROM meter_consumption2;
    LL = %sql SELECT timestamp, SUM(kWh) AS kWh, Phase FROM meter_consumption2 GROUP BY timestamp, Phase
    LL = LL.DataFrame()
    LL.sort_values(by='timestamp', ascending=False, inplace=True)
    LL['Cluster'] = bif['grid_element_id_lower'].iloc[ii]
#    bif.loc[ii,'Meters_in_cluster'] = JJ[0][0]
#    bif.loc[ii,'Total_consumption'] = KK[0][0]
    
    print(bif['grid_element_id_lower'].iloc[ii])
    print(LL.head(48))

/var/folders/03/68xx11110r9b4mpdz752w79r0000gn/T/ipykernel_25011/597468172.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/03/68xx11110r9b4mpdz752w79r0000gn/T/ipykernel_25011/597468172.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



17Y73
                      timestamp       kwh phase Cluster
29970 2024-06-22 01:00:00+00:00  3.727918     C   17Y73
29995 2024-06-22 00:00:00+00:00  3.531852     C   17Y73
29774 2024-06-21 23:00:00+00:00  4.212833     C   17Y73
29990 2024-06-21 22:00:00+00:00  3.499614     C   17Y73
29501 2024-06-21 21:00:00+00:00  3.051680     C   17Y73
30014 2024-06-21 20:00:00+00:00  3.972501     C   17Y73
30399 2024-06-21 19:00:00+00:00  4.354119     C   17Y73
29739 2024-06-21 18:00:00+00:00  3.536770     C   17Y73
29705 2024-06-21 17:00:00+00:00  5.279384     C   17Y73
29926 2024-06-21 16:00:00+00:00  2.592503     C   17Y73
30111 2024-06-21 15:00:00+00:00  1.875825     C   17Y73
29980 2024-06-21 14:00:00+00:00  2.865304     C   17Y73
30012 2024-06-21 13:00:00+00:00  3.170750     C   17Y73
29553 2024-06-21 12:00:00+00:00  4.013144     C   17Y73
30423 2024-06-21 11:00:00+00:00  3.280029     C   17Y73
29672 2024-06-21 10:00:00+00:00  2.937750     C   17Y73
30398 2024-06-21 09:00:00+00:00  4.233155 

L17-136862
                      timestamp        kwh phase     Cluster
29970 2024-06-22 01:00:00+00:00  25.901182     C  L17-136862
29995 2024-06-22 00:00:00+00:00  26.812506     C  L17-136862
29774 2024-06-21 23:00:00+00:00  31.056635     C  L17-136862
29990 2024-06-21 22:00:00+00:00  28.637979     C  L17-136862
29501 2024-06-21 21:00:00+00:00  26.499258     C  L17-136862
30014 2024-06-21 20:00:00+00:00  24.659268     C  L17-136862
30399 2024-06-21 19:00:00+00:00  23.378557     C  L17-136862
29739 2024-06-21 18:00:00+00:00  27.883983     C  L17-136862
29705 2024-06-21 17:00:00+00:00  22.566304     C  L17-136862
29926 2024-06-21 16:00:00+00:00  26.764394     C  L17-136862
30111 2024-06-21 15:00:00+00:00  27.774786     C  L17-136862
29980 2024-06-21 14:00:00+00:00  22.922455     C  L17-136862
30012 2024-06-21 13:00:00+00:00  31.656108     C  L17-136862
29553 2024-06-21 12:00:00+00:00  27.502772     C  L17-136862
30423 2024-06-21 11:00:00+00:00  29.842712     C  L17-136862
29672 2024-06

L17-137896
                      timestamp       kwh phase     Cluster
29970 2024-06-22 01:00:00+00:00  5.689480     C  L17-137896
29995 2024-06-22 00:00:00+00:00  5.182631     C  L17-137896
29774 2024-06-21 23:00:00+00:00  4.783221     C  L17-137896
29990 2024-06-21 22:00:00+00:00  4.798033     C  L17-137896
29501 2024-06-21 21:00:00+00:00  3.850838     C  L17-137896
30014 2024-06-21 20:00:00+00:00  4.214918     C  L17-137896
30399 2024-06-21 19:00:00+00:00  2.464754     C  L17-137896
29739 2024-06-21 18:00:00+00:00  5.708150     C  L17-137896
29705 2024-06-21 17:00:00+00:00  4.427323     C  L17-137896
29926 2024-06-21 16:00:00+00:00  4.339946     C  L17-137896
30111 2024-06-21 15:00:00+00:00  4.391005     C  L17-137896
29980 2024-06-21 14:00:00+00:00  3.427843     C  L17-137896
30012 2024-06-21 13:00:00+00:00  5.286655     C  L17-137896
29553 2024-06-21 12:00:00+00:00  5.179265     C  L17-137896
30423 2024-06-21 11:00:00+00:00  4.240470     C  L17-137896
29672 2024-06-21 10:00:00+00:

In [415]:
grid_element_id

'L17-136043'

In [413]:
GG = %sql select * from grid_get_downstream('{grid_id}', '{grid_element_id}') limit20;

In [414]:
GG

grid_id,grid_element_id,type,customer_type,phases,is_underground,is_producer,is_consumer,is_switchable,switch_is_open,terminal1_cn,terminal2_cn,power_flow_direction,upstream_grid_element_id,geometry,meta
North Central Zone,115412,ACLineSegment,arc:SECONDARY_CONDUCTOR,C,True,False,False,False,None,VCN.2550,VCN.115412,1to2,2550,0102000020E61000000200000093C408A0999754C0E639315EE45D4340877EFF119D9754C01206D4B3E75D4340,"{'name': None, 'label': None, 'system': None, 'rating_a': 205, 'materials': 'Al', 'latitude_1': None, 'latitude_2': None, 'make_model': '1/0 AWGTriplex', 'line_length': None, 'longitude_1': None, 'longitude_2': None, 'is_insulated': None, 'ph_ph_voltage': 240, 'voltage_level': 'LV', 'wire_diameter': 19.812, 'hvmv_parent_element': None, 'is_operation_status': True, 'mvlv_parent_element': None, 'is_ground_deployable': None}"
North Central Zone,15822,ACLineSegment,arc:SECONDARY_CONDUCTOR,C,True,False,False,False,None,VCN.48380,VCN.15822,1to2,48380,0102000020E610000002000000E8C0D6D99D9754C03124CEA4CA5D434015D966CF9A9754C0CFF5B449D25D4340,"{'name': None, 'label': None, 'system': None, 'rating_a': 150, 'materials': 'Al', 'latitude_1': None, 'latitude_2': None, 'make_model': '2 AWGTriplex', 'line_length': None, 'longitude_1': None, 'longitude_2': None, 'is_insulated': None, 'ph_ph_voltage': 240, 'voltage_level': 'LV', 'wire_diameter': 19.812, 'hvmv_parent_element': None, 'is_operation_status': True, 'mvlv_parent_element': None, 'is_ground_deployable': None}"
North Central Zone,17Y18-2,Meter,Meter,C,True,False,True,False,None,VCN.36829,VCN.17Y18-2,1to2,36829,0101000020E61000007AD44D379D9754C0D92A486FC45D4340,"{'name': None, 'label': None, 'sdp_id': None, 'address': '1239 E Fry Rd', 'interval': None, 'latitude': None, 'estimated': None, 'longitude': None, 'lv_feeder': None, 'mv_feeder': 'L17', 'read_type': None, 'tariff_id': None, 'make_model': None, 'meter_type': None, 'is_critical': None, 'meter_number': '84962651', 'zone_level_1': None, 'zone_level_2': None, 'zone_level_3': None, 'zone_level_4': None, 'energy_source': None, 'voltage_level': 'LV', 'maximal_demand': None, 'commission_date': None, 'meter_multiplier': None, 'source_rating_kw': None, 'type_of_consumer': 'residential', 'connected_to_lvmv': None, 'replacement_cycle': None, 'service_drop_type': None, 'hvmv_parent_element': None, 'is_operation_status': True, 'standard_profile_id': None, 'maximal_demand_units': None, 'parent_transformer_id': None, 'business_identifier_id': '1141701'}"
North Central Zone,17Y18-2ZA,Meter,Meter,C,True,False,True,False,None,VCN.15822,VCN.17Y18-2ZA,1to2,15822,0101000020E610000015D966CF9A9754C0CFF5B449D25D4340,"{'name': None, 'label': None, 'sdp_id': None, 'address': '6632 Sweet Wood Ct', 'interval': None, 'latitude': None, 'estimated': None, 'longitude': None, 'lv_feeder': None, 'mv_feeder': 'L17', 'read_type': None, 'tariff_id': None, 'make_model': None, 'meter_type': None, 'is_critical': None, 'meter_number': '87439250', 'zone_level_1': None, 'zone_level_2': None, 'zone_level_3': None, 'zone_level_4': None, 'energy_source': None, 'voltage_level': 'LV', 'maximal_demand': None, 'commission_date': None, 'meter_multiplier': None, 'source_rating_kw': None, 'type_of_consumer': 'residential', 'connected_to_lvmv': None, 'replacement_cycle': None, 'service_drop_type': None, 'hvmv_parent_element': None, 'is_operation_status': True, 'standard_profile_id': None, 'maximal_demand_units': None, 'parent_transformer_id': None, 'business_identifier_id': '1059411'}"
North Central Zone,17Y18-3ZA,Meter,Meter,C,True,False,True,False,None,VCN.115412,VCN.17Y18-3ZA,1to2,115412,0101000020E6100000877EFF119D9754C01206D4B3E75D4340,"{'name': None, 'label': None, 'sdp_id': None, 'address': '2713 Sweet Cider Rd', 'interval': None, 'latitude': None, 'estimated': None, 'longitude': None, 'lv_feeder': None, 'mv_feeder': 'L17', 'read_type': None, 'tariff_id': None, 'make_model': None, 'meter_type': None, 'is_critical': None, 'meter_number': '13890496', 'zone_level_1': 